# Model: Combined - LSTM for (merged) text + Dense for others

Our NN models vary in the way we preprocess the text features (most important feature) towards feeding it into our NN.
This also effects the architechture of the NN.

In this notebook we:
1. Merge all text features (Title, Description, params) into one feature.
2. We learn the representation of the text feature by an LSTM (its output vector is the text representation).
3. We combine the LSTMed text with all other features.

In [2]:
%load_ext autoreload

In [3]:
%run stephan_modules.ipynb

/glob/intel-python/versions/2018/intelpython3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [ ]:
DATA_PATH = '/home/u14303/Avito'
HELPER_DATA_PATH = '/home/u14303/Avito/helper_data'

## Feature enrichment

In [ ]:
print('loading data...')
train, test = load_data(DATA_PATH)
train, test = basic_enrichment(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = load_image_features(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = load_text_features(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = add_aggregated_features(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = numeric_features_cleaning(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = complete_image_top_1(train, test, helper_data_path=HELPER_DATA_PATH)
train, test = complete_price(train, test, helper_data_path=HELPER_DATA_PATH)

loading data...


In [ ]:
list(train.columns)

## Vectorize features towards input to an NN

In [ ]:
# text_features = ['title', 'description', 'param123']
text_feature = 'title_description_params'
cat_features = ['user_type', \
                'region', 'city', \
                'parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', \
                'image_top_1_class', 'image_top_1_rounded_regression', \
                'month', 'day', 'weekday', \
                'has_price', 'has_description', 'has_params', 'has_image']
cont_ord_features = ['image_top_1_regression', \
                     'log_price_regression', \
                     'avg_days_up_user', 'avg_times_up_user', 'n_user_items', 'user_ads_count', \
                     'log_item_seq_number', \
                     'img_size', 'img_luminance', 'img_colorfulness', 'img_confidence', 'log_img_sharpness', 'log_img_keypoints', \
                     'title_word_count', 'description_word_count', 'merged_params_word_count', \
                     'description_non_regular_chars_ratio', 'title_capital_letters_ratio','description_capital_letters_ratio', \
                     'title_non_regular_chars_ratio', 'title_adj_to_len_ratio', 'title_noun_to_len_ratio',\
                     'title_sentiment']

train_y_prob = train['deal_probability']

## Preprocess text towards input to an LSTM

Import stopwords.

In [ ]:
import codecs

stopwords = set()
with codecs.open(('stopwords_ru.txt'), encoding='cp1251') as ins:
    for w in ins:
        word = w.strip("\r\n")
        word = word.strip("\n")
        stopwords.add(word.lower())

In [ ]:
Tokenize and Vectorize (keras encoded one-hot representation (each onehot vec represented as an int number)) text feature.

See: https://keras.io/preprocessing/text/#one_hot

In [ ]:
from keras.preprocessing.text import one_hot, Tokenizer, text_to_word_sequence

# Those consts are important for the NN itself
max_words_in_desc_title_param = 150 # See text analysis notebook. 95% are shorter than a 110.
word_embed_dim = 300

train_x_text = train[[text_feature]]
test_x_text = test[[text_feature]]
train_x_text[text_feature] = train_x_text[text_feature].str.lower()
test_x_text[text_feature] = test_x_text[text_feature].str.lower()
tokenizer = Tokenizer(num_words = max_words_in_desc_title_param)
all_text = np.hstack([train_x_text[text_feature], test_x_text[text_feature]])

print('Tokenizing text...')
tokenizer.fit_on_texts(all_text)
print('Done tokenizing.')

del all_text
gc.collect()

In [ ]:
# By default text_to_word_sequence automatically does 4 things:
#   Splits words by space (split=” “), 
#   Filters out punctuation (filters=’!”#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n’).
#   Converts text to lowercase (lower=True).
# We add stopwords filtering to the process.

def my_text_to_word_sequence(text):
    result = []
    for word in text_to_word_sequence(text):
        if word not in stopwords:
            result.append(word)
    return result

In [ ]:
print('Applying tokenizer on text...')

train_x_text[text_feature] = train_x_text.apply(lambda r: [tokenizer.word_index[word] for word in my_text_to_word_sequence(r[text_feature])], axis=1)
test_x_text[text_feature] = test_x_text.apply(lambda r: [tokenizer.word_index[word] for word in my_text_to_word_sequence(r[text_feature])], axis=1)
    
train_x_text = pad_sequences(train_x_text[text_feature], maxlen=max_words_in_desc_title_param)
test_x_text = pad_sequences(test_x_text[text_feature], maxlen=max_words_in_desc_title_param)

print('Done.')

Import and load FastText (Facebook's) Russian wikipedia word embeddings.

In [ ]:
def get_fasttext_embedding_matrix(data_path, tokenizer, embedding_dim):
    print('loading embeddings...')
    
    EMBEDDING_FILE_PATH = os.path.join(data_path, 'cc.ru.300.vec')
    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE_PATH))

    vocab_size = len(tokenizer.word_index) + 2
    embedding_matrix = np.random.rand(vocab_size, embedding_dim)
    
    print('creating embedding matrix...')
    embedding_exists = 0
    no_embeddings = 0
    for word, i in tokenizer.word_index.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
            embedding_exists += 1
        else:
            no_embeddings += 1
    
    print ("There are total of {} words in our corpus.".format(embedding_exists+no_embeddings))
    print ("There are {} embeddings in FastText.".format(len(embeddings_index)))
    print ("We have embeddings for {} words ({}% existing embeddings).".format(embedding_exists, \
                                                                               (100*embedding_exists/(embedding_exists+no_embeddings))))
    print ("Embedding is missing for {} words.".format(no_embeddings))

    del embeddings_index
    gc.collect()
    
    print('done loading embeddings...')
    return embedding_matrix, vocab_size

In [ ]:
embedding_matrix, vocab_size = get_fasttext_embedding_matrix(data_path=DATA_PATH, \
                                                             tokenizer=tokenizer, embedding_dim = word_embed_dim)

#### Categorical features.

Vectorize all loaded categorical features.


See: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
train_x_cat = train[cat_features]
test_x_cat = test[cat_features]
for col in cat_features:
    train_x_cat[col] = train_x_cat[col].astype('category')
    test_x_cat[col] = test_x_cat[col].astype('category')

# Encode to integers.
# For vectorization (encoding) we concat both train and test into one
all_cat = pd.concat([train_x_cat, test_x_cat], axis = 0)
for col in cat_features:
    enc = preprocessing.LabelEncoder().fit(all_cat[col])
    train_x_cat[col] = enc.transform(train_x_cat[col])
    test_x_cat[col] = enc.transform(test_x_cat[col])

# One-hot encode:
enc = OneHotEncoder()
enc.fit(pd.concat([train_x_cat, test_x_cat], axis = 0))
train_x_cat = enc.transform(train_x_cat)
test_x_cat = enc.transform(test_x_cat)

del all_cat
gc.collect()

#### Numerical(Continious/Ordinal) features

Normalize all loaded numeric features.

In [ ]:
train_x_numerical = train[cont_ord_features]
test_x_numerical = test[cont_ord_features]
train_x_numerical.fillna(0, inplace = True)
test_x_numerical.fillna(0, inplace = True)
for col in cont_ord_features:
    train_x_numerical[col] = train_x_numerical[col].astype('float64')
    test_x_numerical[col] = test_x_numerical[col].astype('float64')

# Normalize features:
train_x_numerical = normalize(train_x_numerical, axis=0)
test_x_numerical = normalize(test_x_numerical, axis=0)

# Learning - Neural Net

Here we should have a pad sequence out of the text features
All categorials should be onehot encoded
All continious/ordinal should be vectorized

### Define network

TODO(): Try hidden_dim1 = 512, hidden_dim2 = 128 after loading FastText

In [ ]:
lstm_out_dim = 256
cat_hidden_dim = 128
merged_hidden_dim = 256
out_dim = 1

Function that gets a tokenizer, and returns fasttext_embedding_matrix that will be loaded into the embedding layer of our NN

In [ ]:
# LSTM
l_text_input = Input(shape=(max_words_in_desc_title_param,), name="text_lstm_input")
# Load FastText's weights:
l_text_embedding = Embedding(input_dim=vocab_size, output_dim=word_embed_dim, weights = [embedding_matrix], \
                             input_length=max_words_in_desc_title_param, name='text_embeddings')(l_text_input)
lstm_out = LSTM(lstm_out_dim, name='LSTM', dropout=0.2, recurrent_dropout=0.2)(l_text_embedding)

# Categoricals
l_cat_input = Input(shape=(train_x_cat.shape[1],), sparse=True, name="cat_input")
l_hidden_cat = Dense(cat_hidden_dim, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-6), name='l_hidden_cat')(l_cat_input)

# Numerical
l_numerical_input = Input(shape=(train_x_numerical.shape[1],), name="numerical_input")

# Aggregate all inputs into one hidden layer.
l_aggregative = concatenate([l_hidden_text, l_hidden_cat, l_numerical_input])

l_merged_hidden = Dense(merged_hidden_dim, activation='relu',
                            kernel_regularizer=regularizers.l2(1e-6), name='l_merged_hidden')(l_aggregative)

output = Dense(out_dim, activation='sigmoid', name='output')(l_merged_hidden) # This is the logistic regression output

rmsprop_opt = RMSprop(lr=0.00001) # Best for training RNNs.
def rmse_err(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

model = Model(inputs=[l_text_input, l_cat_input, l_numerical_input], outputs=main_output)
model.compile(optimizer=rmsprop_opt,
              ,loss=[rmse_err]) # Higher weight for main output.
print(model.summary())

load = False
fname = 'NN-LSTM-COMBINED-MERGED-model-weights.h5'
if load:
    model.load_weights(fname)

In [ ]:
reduce_lr_cd = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)
save_weights_cd = ModelCheckpoint(fname, save_weights_only=True, mode='auto', period=1)

 = 256

# We want to double batch size every two epochs.
nepochs = 4
for i in range(nepochs):
    print ('Curr batch size: {}'.format(nbatch_size))
    model.fit(train_x_text, train_y_prob, 
              validation_split = 0.15, 
              epochs=2, 
              batch_size=nbatch_size,
              callbacks=[reduce_lr_cd, save_weights_cd])
    nbatch_size *= 2

### Save predictions to futher ensambels

In [ ]:
y_pred_train = model.predict({'text_lstm_input': train_x_text, 'cat_input': train_x_cat, 'numerical_input': train_x_numerical})
y_pred_test = model.predict({'text_lstm_input': test_x_text, 'cat_input': test_x_cat, 'numerical_input': test_x_numerical})

In [ ]:
res_train_df = pd.DataFrame(train['item_id'])
res_train_df['deal_probability'] = y_pred_train

In [ ]:
res_test_df = pd.DataFrame(test['item_id'])
res_test_df['deal_probability'] = y_pred_test

In [ ]:
res_df = pd.concat([res_train_df, res_test_df], axis = 0)

In [ ]:
res_df.to_csv('/home/u14303/Avito/Predictions/NN-LSTM-COMBINED-MERGED.csv.gz', index=False, compression='gzip')